# Capstone

Michael Schillawski, 10 April 2018

Data Science Immersive, Generaly Assembly

## Imports

In [1]:
import os
import json
import re
import string
import multiprocessing
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.io.json import json_normalize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed


%matplotlib inline

os.getcwd()

'/Users/mjschillawski/Google Drive/Data/generalassembly/projects/GitHub Portfolio/capstone_project'

## Load Data

In [75]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/'
file = 'data_records_27638.txt'

data = pd.read_table(path+file,header=None,names=['recipe'],index_col=1)

In [76]:
data.head()

,recipe
1,Mushroom-risotto-352492
2,Filipino-bbq-pork-skewers-352163
3,Mushroom-and-Roasted-Garlic-Risotto-525026
4,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...
5,Delicious-Grilled-Hamburgers-Allrecipes


In [77]:
data.tail()

,recipe
27634,Saltimbocca-alla-Romana-824004
27635,Carrotini-MyRecipes-241041
27636,Brioche-531830
27637,Mexican-Hot-Chocolate-Doughnuts-501510
27638,Thai-Curry-Burrito-Bowl-with-Sweet-and-Spicy-T...


In [79]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/metadata27638/'

recipes = []

for i in data.index:
    num = str(i)
    while len(num) < 5:
        num = '0' + num
        
    # https://stackoverflow.com/questions/28373282/how-to-read-a-json-dictionary-type-file-with-pandas
    with open(path+'meta'+num+'.json') as json_data:
        recipe = json.load(json_data)
        recipes.append(recipe)

recipes = json_normalize(recipes)   
recipes

,attributes.course,attributes.cuisine,attributes.holiday,attribution.html,attribution.logo,attribution.text,attribution.url,cookTime,cookTimeInSeconds,flavors.Bitter,...,nutritionEstimates,prepTime,prepTimeInSeconds,rating,source.sourceDisplayName,source.sourceRecipeUrl,source.sourceSiteUrl,totalTime,totalTimeInSeconds,yield
0,[Side Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom Risotto recipes: information powered ...,http://www.yummly.com/recipe/Mushroom-risotto-...,NaN,NaN,NaN,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2009/10/risotto-is-...,http://www.skinnytaste.com,30 minutes,1800.0,servings: 6
1,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Filipino...,http://static.yummly.com/api-logo.png,Filipino BBQ Pork Skewers recipes: information...,http://www.yummly.com/recipe/Filipino-bbq-pork...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2008/08/filipino-bb...,http://www.skinnytaste.com,40 min,2400.0,None
2,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom and Roasted Garlic Risotto recipes: i...,http://www.yummly.com/recipe/Mushroom-and-Roas...,NaN,NaN,1.0000,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/mushroom-roast...,http://www.myrecipes.com,1 Hr 25 Min,5100.0,Serves 6 (serving size: about 1 cup)
3,[Side Dishes],"[French, American]",NaN,<a href='http://www.yummly.com/recipe/Gratin-D...,http://static.yummly.com/api-logo.png,Gratin Dauphinois (Scalloped Potatoes with Che...,http://www.yummly.com/recipe/Gratin-Dauphinois...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/gratin-dauphin...,http://www.myrecipes.com,55 min,3300.0,7 servings (serving size: 1 cup)
4,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Deliciou...,http://static.yummly.com/api-logo.png,Delicious Grilled Hamburgers recipes: informat...,http://www.yummly.com/recipe/Delicious-Grilled...,10 Min,600.0,0.1667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",5 Min,300.0,4,AllRecipes,http://allrecipes.com/Recipe/delicious-grilled...,http://www.allrecipes.com,15 Min,900.0,3 servings
5,[Main Dishes],[American],"[Winter, Christmas, Thanksgiving]",<a href='http://www.yummly.com/recipe/Hickory_...,http://static.yummly.com/api-logo.png,Hickory-Smoked Bourbon Turkey recipes: informa...,http://www.yummly.com/recipe/Hickory_smoked-Bo...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/hickory-smoked...,http://www.myrecipes.com,None,NaN,Makes 12 to 14 servings
6,[Main Dishes],[American],NaN,<a href='http://www.yummly.com/recipe/Marinate...,http://static.yummly.com/api-logo.png,Marinated Beef Tenderloin recipes: information...,http://www.yummly.com/recipe/Marinated-Beef-Te...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",25 Min,1500.0,3,Martha Stewart,http://www.marthastewart.com/962284/marinated-...,http://www.marthastewart.com,3 Hr 30 Min,12600.0,servings: 8
7,[Desserts],[Italian],[Summer],<a href='http://www.yummly.com/recipe/Cherry-T...,http://static.yummly.com/api-logo.png,Cherry Tortoni recipes: information powered by...,http://www.yummly.com/recipe/Cherry-Tortoni-My...,NaN,NaN,0.3333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/cherry-tortoni...,http://www.myrecipes.com,None,NaN,10 servings (serving size: 1 [3-inch] tortoni ...
8,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Crock-Po...,http://static.yummly.com/api-logo.png,Crock-Pot Italian Turkey Meatballs and Homemad...,http://www.yummly.com/recipe/Crock-Pot-Italian...,NaN

In [81]:
recipes.to_csv('assets/recipes_dataset.csv')

## EDA

In [4]:
recipes = pd.read_csv('assets/recipes_dataset.csv',index_col=0)

# transform ingredient field back into list when importing from CSV
recipes['ingredientLines'] = recipes['ingredientLines'].apply(
    lambda x: [item for item in x.split('\'') if item not in ('\,','[',']',', ')])

recipes.head()

In [ ]:
# distribution of courses

In [ ]:
# distribution of cuisines

### Extract columns we care about

In [50]:
short_recipes = recipes[['attributes.course','attributes.cuisine','name','ingredientLines']]

In [51]:
short_recipes.to_csv('assets/short_recipes.csv')

In [52]:
short_recipes['ingredientLines'][17464]

['9 ounces farro perlato (semi-pearled emmer), about 1 1/3 cups uncooked (see substitutes below)',
 '1 teaspoon chia seed or ground flaxseeds',
 '3 medium zucchini, trimmed and halved lengthwise',
 '1/2 red bell pepper, seeded',
 '12-16 ounces cherry tomatoes, halved',
 '2 green onions, green parts only, thinly sliced',
 '2 tablespoons white balsamic or white wine vinegar',
 '1 tablespoon lemon juice',
 '2 teaspoons spicy or Dijon mustard, or to taste',
 '1 large clove garlic, minced or pressed',
 '1/4 teaspoon salt (optional)',
 '1/8 teaspoon freshly ground black pepper',
 '1/4 cup chopped fresh basil',
 'Salt and black pepper, to taste',
 'Additional lemon juice, to taste']

## Natural Language Processing

strip numbers

strip common measurements and their abbreviations

strip punctuation

strip preparation methods

strip 'to taste'

inventory my spice rack

strip spices / pantry items

### Preprocessing

In [53]:
# single-threaded
def process_ingredients(recipes,nondescript=0,drop_words=None):
    # create the patterns

    # import punctuation characters
    # to remove all punctuation
    punct = string.punctuation
    punct_pattern = r"[{}]".format(punct)

    # to remove all numbers
    number_pattern = r"\d+\s"
    
    # embedded numbers
    embed_num_pattern = r".\d+."
    
    # removed prep methods
    prep_pattern = r"[a-z]+ed"
    
    # strip pluralization
    plural_pattern = r"s\s"
    recipes_ingredients = []

    for recipe in recipes:
        ingredients = []
    
        for item in recipe:

            # strip punctuation
            text = re.sub(punct_pattern," ",item)
            # strip standalone numbers
            text = re.sub(number_pattern,"",text)
            # strip embedded numbers
            text = re.sub(embed_num_pattern,"",text)
            # strip preparation methods
            text = re.sub(prep_pattern,"",text)
            # strip pluralization
            text = re.sub(plural_pattern," ",text)
            
            # tokenize
            tokenizer = RegexpTokenizer(r'\w+')
            processed_text = tokenizer.tokenize(text)

            # remove stop words
            processed_text = [text.lower() for text in processed_text if text.lower() 
                              not in stopwords.words('english')]
            
            # remove non-descript recipe words
            if nondescript == 1 and drop_words != None:
                processed_text = [text.lower() for text in processed_text if text.lower()
                                 not in drop_words]
                
            # append all each list that to describe an ingredient of the recipe
            ingredients.append(processed_text)

        # joined space-separated strings
        # attach all modifiers that describe each ingredient (non-separated)
        clean_ingredients = [" ".join(word) for word in ingredients]

        # append all ingredients for each recipe
        recipes_ingredients.append(clean_ingredients)    
    
    recipes_ingredients = [" ".join(ingredient) for ingredient in recipes_ingredients]
    recipes_ingredients = pd.DataFrame(recipes_ingredients)
    return recipes_ingredients

In [54]:
process_ingredients(short_recipes['ingredientLines'][:5])

,0
0,cup baby bella mushroom cup arborio rice tsp o...
1,lb pork country style rib fat cut x cubes oz c...
2,whole garlic heads tablespoon plu teaspoon ext...
3,garlic clove cooking spray potatoe cut inch sl...
4,pound lean ground beef tablespoon worcestershi...


In [55]:
# multi-threaded
def multi_process_ingredients(recipes,join=1,nondescript=0,drop_words=None):
    # create the patternsu

    # import punctuation characters
    # to remove all punctuation
    punct = string.punctuation
    punct_pattern = r"[{}]".format(punct)

    # to remove all numbers
    number_pattern = r"\d+\s"

    # embedded numbers
    embed_num_pattern = r".\d+."
    
    # removed prep methods
    prep_pattern = r"[a-z]+ed"
    
    # strip pluralization
    plural_pattern = r"s\s"
    
    # strip -ly
    ly_pattern = r"[a-z]+ly"
    
    recipes_ingredients = []
    ingredients = []

    for item in recipes:

        # strip punctuation
        text = re.sub(punct_pattern," ",item)
        # strip standalone numbers
        text = re.sub(number_pattern,"",text)
        # strip embedded numbers
        text = re.sub(embed_num_pattern,"",text)
        # strip preparation methods
        text = re.sub(prep_pattern,"",text)
        # strip pluralization
        text = re.sub(plural_pattern," ",text)
        # strip ly
        text = re.sub(ly_pattern,"",text)

        # tokenize
        tokenizer = RegexpTokenizer(r'\w+')
        processed_text = tokenizer.tokenize(text)

        # remove stop words
        processed_text = [text.lower() for text in processed_text if text.lower() 
                          not in stopwords.words('english')]

        # remove non-descript recipe words
        if nondescript == 1 and drop_words != None:
            processed_text = [text.lower() for text in processed_text if text.lower()
                             not in drop_words]

        # append all each list that to describe an ingredient of the recipe
        ingredients.append(processed_text)

    # joined space-separated strings
    # attach all modifiers that describe each ingredient (non-separated)
    clean_ingredients = [" ".join(word) for word in ingredients]

    # append all ingredients for each recipe
    recipes_ingredients.append(clean_ingredients)    
    
    if join == 0:
        pass
    else:
        recipes_ingredients = [" ".join(ingredient) for ingredient in recipes_ingredients]
    
    return recipes_ingredients

In [56]:
num_cores = multiprocessing.cpu_count()
inputs = short_recipes['ingredientLines']

if __name__ == "__main__":
    recipes = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i) for i in inputs)

In [57]:
recipes[0]

['cup baby bella mushroom cup arborio rice tsp olive oil tsp butter shallot cup white wine cup fat free chicken stock vegetable stock salt pepper salt pepper cup parmesan cheese tbsp parsley']

In [58]:
# 1 list of ingredients for each recipe
recipes = [" ".join(recipe) for recipe in recipes]
recipes = pd.DataFrame(recipes)

In [59]:
recipes.head()

,0
0,cup baby bella mushroom cup arborio rice tsp o...
1,lb pork country style rib fat cut x cubes oz c...
2,whole garlic heads tablespoon plu teaspoon ext...
3,garlic clove cooking spray potatoe cut inch sl...
4,pound lean ground beef tablespoon worcestershi...


### Word Counts for Custom Stop Word Dictionary

In [60]:
# word counts
# get the words that occur most often in recipes
# these are candidates for removal in order to simplify the axis that we compare recipes

cvec = CountVectorizer(strip_accents=ascii)
cvecdata = cvec.fit_transform(recipes[0])

cvec_dense  = pd.DataFrame(cvecdata.todense(),
             columns=cvec.get_feature_names())

word_count = cvec_dense.sum(axis=0)    
cw = word_count.sort_values(ascending = False)
print(cw[0:100])

cw_dict = dict(cw)

cup           76725
teaspoon      46142
tablespoon    40078
pepper        26446
salt          26439
fresh         22512
ounce         18745
ground        18002
oil           17914
garlic        13556
onion         13464
pound         11796
cheese        11621
tsp           11280
olive         11156
large         10407
black         10156
sauce          9877
sugar          9866
clove          9576
cut            9491
chicken        8941
inch           8080
tbsp           7973
green          7690
xbd            7629
powder         7565
butter         7549
juice          7408
taste          7376
              ...  
dry            3032
pieces         2910
slice          2907
beef           2889
brown          2881
sour           2874
parmesan       2859
bell           2770
cooking        2770
paste          2728
frozen         2575
mustard        2573
italian        2557
vanilla        2546
chile          2541
garnish        2523
yellow         2510
oregano        2474
thick          2418


In [61]:
len(cw)

9712

In [76]:
# quick function to manually evaluate words that ought to be removed
# https://stackoverflow.com/questions/5844672/delete-an-item-from-a-dictionary

def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

def eval_words(word_list):
    keeps = []
    nondescript = []
    
    nondescript_words = [] 
    keep_words = []
    
    for key,value in word_list.items():
            word_eval = input('Keep {}: {}, y or n?'.format(key,value))
        
            if word_eval == 'n':
                nondescript_words.append(key)
            else:
                keep_words.append(key)
            
            remaining_list = removekey(word_list,key)
            
            if len(nondescript_words) % 100 == 0:
                nondescript = nondescript + nondescript_words
                keeps = keeps + keep_words
                
                # empty holding lists
                keep_words = []
                nondescript_words = []
                
                prompt_continue = input('Continue: yes or no?')
                if prompt_continue == "yes":
                    pass
                else:
                    # export lists as pickles for recovery
                    # store outside the environment to limit reprocessing
                    words_lists = (keeps,nondescript,remaining_list)
                    names = ("keeps","nondescript","remaining")
                    for index,word in enumerate(words_lists):
                        with open("assets/"+names[index]+".pickle","wb") as file:
                            pickle.dump(word,file)
                    return keeps, nondescript,remaining_list
    
    # export word lists for recovery
    # so we don't have to do this multiple times
    words_lists = (keeps,nondescript,remaining_list)
    names = ("keeps","nondescript","remaining")
    for index,word in enumerate(words_lists):
        with open("assets/"+names[index]+".pickle","wb") as file:
            pickle.dump(word,file)
    
    return keep_words, nondescript_words, remaining_list

In [77]:
keep, drop, remain = eval_words(cw)

Keep cup: 76725, y or n?n
Keep teaspoon: 46142, y or n?n
Keep tablespoon: 40078, y or n?n
Keep pepper: 26446, y or n?y
Keep salt: 26439, y or n?y
Keep fresh: 22512, y or n?n
Keep ounce: 18745, y or n?n
Keep ground: 18002, y or n?y
Keep oil: 17914, y or n?y
Keep garlic: 13556, y or n?y
Keep onion: 13464, y or n?y
Keep pound: 11796, y or n?n
Keep cheese: 11621, y or n?y
Keep tsp: 11280, y or n?n
Keep olive: 11156, y or n?y
Keep large: 10407, y or n?n
Keep black: 10156, y or n?y
Keep sauce: 9877, y or n?y
Keep sugar: 9866, y or n?y
Keep clove: 9576, y or n?y
Keep cut: 9491, y or n?n
Keep chicken: 8941, y or n?y
Keep inch: 8080, y or n?n
Keep tbsp: 7973, y or n?n
Keep green: 7690, y or n?y
Keep xbd: 7629, y or n?n
Keep powder: 7565, y or n?y
Keep butter: 7549, y or n?y
Keep juice: 7408, y or n?y
Keep taste: 7376, y or n?n
Keep oz: 7168, y or n?n
Keep white: 6822, y or n?y
Keep cream: 6440, y or n?y
Keep flour: 6256, y or n?y
Keep water: 6144, y or n?y
Keep small: 5871, y or n?n
Keep ium: 5

Keep dice: 489, y or n?y
Keep quart: 489, y or n?n
Keep french: 484, y or n?y
Keep pizza: 479, y or n?y
Keep broccoli: 476, y or n?y
Keep garnishe: 475, y or n?n
Keep total: 471, y or n?n
Keep good: 471, y or n?n
Keep split: 467, y or n?n
Keep old: 462, y or n?n
Keep dash: 457, y or n?n
Keep bought: 457, y or n?n
Continue: yes or no?yes
Keep chives: 456, y or n?y
Continue: yes or no?yes
Keep salmon: 455, y or n?y
Continue: yes or no?yes
Keep salz: 454, y or n?y
Continue: yes or no?yes
Keep queso: 454, y or n?y
Continue: yes or no?yes
Keep crumbs: 453, y or n?n
Keep kernel: 452, y or n?n
Keep romaine: 450, y or n?y
Keep grape: 446, y or n?y
Keep skim: 446, y or n?y
Keep soup: 440, y or n?y
Keep romano: 438, y or n?y
Keep well: 437, y or n?n
Keep raw: 436, y or n?n
Keep pint: 434, y or n?n
Keep feta: 432, y or n?y
Keep chiles: 429, y or n?y
Keep regular: 423, y or n?n
Keep olives: 418, y or n?y
Keep root: 417, y or n?y
Keep cornmeal: 417, y or n?y
Keep spicy: 415, y or n?y
Keep halve: 41

Keep sprouts: 164, y or n?y
Keep bund: 163, y or n?n
Keep basilikum: 163, y or n?n
Keep xeeche: 162, y or n?n
Keep nonstick: 161, y or n?n
Keep sunflower: 161, y or n?y
Keep dose: 160, y or n?n
Continue: yes or no?yes
Keep crema: 160, y or n?y
Continue: yes or no?yes
Keep tartar: 160, y or n?y
Continue: yes or no?yes
Keep avocados: 159, y or n?y
Continue: yes or no?yes
Keep big: 159, y or n?n
Keep blueberries: 159, y or n?y
Keep rings: 158, y or n?n
Keep cherrie: 157, y or n?y
Keep available: 157, y or n?n
Keep dish: 157, y or n?n
Keep japanese: 155, y or n?y
Keep lower: 155, y or n?n
Keep wei: 155, y or n?n
Keep chorizo: 154, y or n?y
Keep watercres: 153, y or n?y
Keep liqueur: 153, y or n?y
Keep cm: 151, y or n?n
Keep knoblauchzehe: 151, y or n?n
Keep linguine: 150, y or n?y
Keep great: 149, y or n?n
Keep tail: 148, y or n?y
Keep table: 148, y or n?n
Keep crabmeat: 148, y or n?y
Keep pearl: 147, y or n?y
Keep spread: 147, y or n?y
Keep boston: 146, y or n?y
Keep petite: 146, y or n?y

In [117]:
keep, drop, remain = eval_words(cw[854:])

Keep coleslaw: 93, y or n?y
Continue: yes or no?yes
Keep jicama: 93, y or n?y
Continue: yes or no?yes
Keep indian: 92, y or n?y
Continue: yes or no?yes
Keep fluid: 92, y or n?n
Keep creme: 92, y or n?y
Keep knoblauchzehen: 91, y or n?n
Keep peach: 91, y or n?y
Keep vegetarian: 91, y or n?n
Keep harina: 91, y or n?y
Keep oranges: 91, y or n?y
Keep basic: 91, y or n?n
Keep rolls: 91, y or n?y
Keep qt: 91, y or n?n
Keep snapper: 91, y or n?y
Keep preserves: 91, y or n?y
Keep hellmann: 90, y or n?n
Keep glaze: 90, y or n?y
Keep without: 89, y or n?n
Keep real: 89, y or n?n
Keep gourmet: 89, y or n?n
Keep tortellini: 88, y or n?y
Keep strawberry: 88, y or n?y
Keep belgian: 88, y or n?y
Keep safflower: 88, y or n?y
Keep tilapia: 88, y or n?y
Keep non: 87, y or n?n
Keep dairy: 87, y or n?y
Keep petersilie: 87, y or n?n
Keep geschnitten: 87, y or n?n
Keep whatever: 87, y or n?n
Keep hour: 87, y or n?n
Keep frosting: 87, y or n?y
Keep stir: 87, y or n?n
Keep wish: 86, y or n?n
Keep proces: 86, 

Keep capsicum: 55, y or n?y
Keep base: 55, y or n?n
Keep ring: 54, y or n?n
Keep evtl: 54, y or n?n
Keep spear: 54, y or n?y
Keep wood: 54, y or n?y
Keep minutes: 54, y or n?n
Keep several: 54, y or n?n
Keep hunt: 54, y or n?n
Keep pinot: 54, y or n?y
Keep trout: 54, y or n?y
Keep farfalle: 54, y or n?y
Keep gelatin: 54, y or n?y
Keep summer: 54, y or n?y
Keep blanc: 54, y or n?y
Keep geriebener: 53, y or n?n
Keep cereal: 53, y or n?y
Keep oat: 53, y or n?y
Keep frische: 53, y or n?y
Keep xfcne: 53, y or n?n
Keep la: 53, y or n?n
Keep squid: 53, y or n?y
Keep hickory: 53, y or n?y
Keep vert: 53, y or n?y
Keep market: 53, y or n?n
Keep medal: 53, y or n?n
Keep version: 53, y or n?n
Keep drain: 52, y or n?n
Keep cover: 52, y or n?n
Keep sure: 52, y or n?n
Keep iient: 52, y or n?n
Keep mortar: 52, y or n?n
Keep flower: 52, y or n?y
Keep port: 52, y or n?y
Keep mirin: 52, y or n?y
Keep oelek: 52, y or n?n
Keep rotel: 52, y or n?n
Keep miso: 51, y or n?y
Keep hatch: 51, y or n?y
Keep marmal

In [118]:
print("keep: {}, drop: {}, remaining: {}".format(len(keep),len(drop),len(remain)))

keep: 279, drop: 300, remaining: 8857


In [122]:
854 + len(keep)+len(drop)

1433

In [124]:
cw[1433:]

kaiser              36
wasabi              36
loave               36
toast               36
beets               36
savoy               36
soybean             36
schwarze            36
er                  36
2lb                 36
guanciale           36
fuji                36
multigrain          35
cabernet            35
maraschino          35
barbeque            35
six                 35
currypaste          35
pt                  35
hing                35
candie              35
frischer            35
mission             35
bones               35
inner               35
instead             35
piecrust            35
essig               35
commercial          35
sausages            35
                    ..
metric               1
mett                 1
meunster             1
mexcian              1
mexicanstyle         1
mexikanische         1
mezzi                1
mgl                  1
miao                 1
menu                 1
mentuccia            1
memphi               1
mehrkornbag

In [107]:
type(cw)

pandas.core.series.Series

In [120]:
# read in pickled results
names = ("1_keeps","1_nondescript","1_remaining")
word_list = []
for name in names:
    with open("assets/"+name+".pickle",'rb') as file_handle:
        word_list.append(pickle.load(file_handle))

### Reprocessing for Recommender

I think I need to process ingredients 2 ways. As I did it above, it's fine. I lump all the ingredients together in order to run it through countvectorizer. 

When I prep data for the recommendation engine, I think I need to treat each ingredient, rather than each word as the unit of comparison. Still subject to analysis. I can probably stand it up as.

In [173]:
# re-process ingredient list, this time removing the non-descript words identified above
# getting data ready for recommender

num_cores = multiprocessing.cpu_count()
inputs = short_recipes['ingredientLines']

if __name__ == "__main__":
    recipes_drops = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i,join=0,
                                                                            nondescript=1,
                                                                            drop_words=word_list[1])
                                                                            for i in inputs)

In [193]:
# drop out ingredients that have been reduced to blanks after stop words

recipes_drops = [[[ingredient for 
                   ingredient in recipe if ingredient != ''] 
                  for recipe in item] 
                 for item in recipes_drops]

# collapse list of list of ingredients into 
# pipe-separated list of ingredients to feed to tokenizer

recommender_recipes = [["|".join(ingredient) for 
                        ingredient in recipe] 
                       for recipe in recipes_drops]

In [194]:
recommender_recipes = pd.DataFrame(recommender_recipes)
recommender_recipes.rename(columns={0:'ingredients'},inplace=True)

## Recommendation Engine

### Build ingredient matrix using countvectorizer

In [199]:
def pipe_tokenizer(string):
    return string.split("|")

In [200]:
pipe_tokenizer(recommender_recipes['ingredients'][0])

['baby bella mushroom',
 'arborio',
 'oil',
 'shallot',
 'free stock stock',
 'salt pepper',
 'salt pepper']

In [205]:
ingredient_cvector = CountVectorizer(strip_accents=ascii,tokenizer=pipe_tokenizer)
ingredient_cvectordata = ingredient_cvector.fit_transform(recommender_recipes['ingredients'])

ingredient_cvector_dense  = pd.DataFrame(ingredient_cvectordata.todense(),
             columns=ingredient_cvector.get_feature_names())

ingredient_cvector_dense.shape

(27638, 63087)

### Inspect Recipe Vectorization

In [204]:
ingredient_cvector_dense.head()

,'','1,'1 bacon intoinch,'1 cinnamon cassia bark,'1 dar,'1 fresh,'1 herbe de provence,'1 hoisin,'1 lean ground,'1 oil,...,zwiebel n klein gew\xfcrfelt,zwiebel n rot,zwiebel n rot feine halbe ringe,zwiebel n rot gew\xfcrfelt,zwiebel n rote,zwiebel n w\xfcrfeln,zwiebel n wei\xdfe milde s\xfc\xdfe,zwiebeln,zwiebeln ca g,zwiebeln streifen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
ingredient_word_count = ingredient_cvector_dense.sum(axis=0)    
cw = ingredient_word_count.sort_values(ascending = False)
print(cw[0:100])

cw_dict = dict(cw)

tablespoon                9373
garlic                    8516
teaspoon salt             7879
teaspoon                  6855
tablespoon oil            4234
salt pepper               3663
salt                      3413
salt ground pepper        3346
fresh                     3151
tablespoon fresh          3053
oil                       2987
teaspoon ground pepper    2556
pepper                    2493
teaspoon pepper           2320
ounce                     2288
teaspoon ground           1791
eggs                      1465
'tablespoon oil           1351
\xbd                      1298
bay                       1155
teaspoon fresh            1090
teaspoon garlic           1019
'tablespoon                944
ground                     940
beaten                     934
tortillas                  829
'oil                       810
teaspoon oil               799
fresh'                     777
tablespoon'                772
                          ... 
yolks                      321
shrimp  

### Recommender

In [208]:
def mean_center_rows(df):
    return (df.T - df.mean(axis=1)).T

In [209]:
recipes_mc = mean_center_rows(ingredient_cvector_dense)

In [210]:
recipes_mc.isnull().sum().sum()

0

In [211]:
recipes_mc.head()

,'','1,'1 bacon intoinch,'1 cinnamon cassia bark,'1 dar,'1 fresh,'1 herbe de provence,'1 hoisin,'1 lean ground,'1 oil,...,zwiebel n klein gew\xfcrfelt,zwiebel n rot,zwiebel n rot feine halbe ringe,zwiebel n rot gew\xfcrfelt,zwiebel n rote,zwiebel n w\xfcrfeln,zwiebel n wei\xdfe milde s\xfc\xdfe,zwiebeln,zwiebeln ca g,zwiebeln streifen
0,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,...,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111
1,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,...,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063,-0.000063
2,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,...,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206,-0.000206
3,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,...,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111,-0.000111
4,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,...,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095,-0.000095


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(recipes_mc)
recipe_sim = pd.DataFrame(sim_matrix, columns=recipes_mc.index, index=recipes_mc.index)

In [ ]:
import seaborn as sns
sns.heatmap(recipe_sim, annot=True, cmap='coolwarm')

### More EDA

In [ ]:
# distribution of ingredients in recipes

In [ ]:
# most common, least common ingredients
# before and after non-descript word removal

# 

In [ ]:
# figure out input format?
# maybe input prompt
# or url scrape?

In [ ]:
# inventory the differences
# suggest additional ingredients in order to complete both recipes

In [64]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [69]:
fuzz.token_set_ratio("mushrooms","baby bella mushrooms")

100

In [73]:
fuzz.ratio("olive","olive oil")

71

In [74]:
fuzz.ratio("mushrooms","baby bella mushrooms")

62

In [75]:
fuzz.ratio("black pepper","bell pepper")

78